In [101]:
import pandas as pd

import random
import re
import pickle
import nltk 

In [102]:
dataset = pd.read_csv("reviews.csv",sep="\t")
dataset.drop(columns= "Unnamed: 0",inplace = True)
dataset

/home/anubhav/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,IP Address
0,US,20422322,R8MEA6IGAHO0B,B00MC4CED8,217304173,BlackVue DR600GW-PMP,Mobile_Electronics,5,0.0,0.0,N,Y,Very Happy!,"As advertised. Everything works perfectly, I'm...",2015-08-31,211.200.63.206
1,US,40835037,R31LOQ8JGLPRLK,B00OQMFG1Q,137313254,GENSSI GSM / GPS Two Way Smart Phone Car Alarm...,Mobile_Electronics,5,0.0,1.0,N,Y,five star,it's great,2015-08-31,192.17.224.99
2,US,51469641,R2Y0MM9YE6OP3P,B00QERR5CY,82850235,iXCC Multi pack Lightning cable,Mobile_Electronics,5,0.0,0.0,N,Y,great cables,These work great and fit my life proof case fo...,2015-08-31,198.9.2.161
3,US,4332923,RRB9C05HDOD4O,B00QUFTPV4,221169481,abcGoodefg® FBI Covert Acoustic Tube Earpiece ...,Mobile_Electronics,4,0.0,0.0,N,Y,Work very well but couldn't get used to not he...,Work very well but couldn't get used to not he...,2015-08-31,203.46.177.63
4,US,44855305,R26I2RI1GFV8QG,B0067XVNTG,563475445,Generic Car Dashboard Video Camera Vehicle Vid...,Mobile_Electronics,2,0.0,0.0,N,Y,Cameras has battery issues,"Be careful with these products, I have bought ...",2015-08-31,201.159.197.149
5,US,7846966,RY8DDL22YG4R5,B00KA6CCVY,713636156,Aweek® Air Acoustic Earpiece Headset for Baofe...,Mobile_Electronics,3,0.0,1.0,N,Y,"It appears to be good, but I'm still waiting f...","It appears to be good, but I'm still waiting f...",2015-08-31,216.93.144.222
6,US,21299354,R2AT2426ZHFUHH,B00MJCDPM2,754171872,Sentey LS-4460 B-Trek S8 Bluetooth Portable St...,Mobile_Electronics,3,0.0,1.0,N,Y,Didn't love the first one,"First one arrived as a brick. Wouldn't work, ...",2015-08-31,220.211.145.12
7,US,28902968,R3RRXU2R23NMQ9,B00ET5AWBY,508373185,iPad Car Headrest Mount Holder for iPad 2/ iPa...,Mobile_Electronics,5,0.0,0.0,N,Y,Five Stars,Worked great for vacation,2015-08-31,192.183.31.50
8,US,2074438,R250PR8VJUZ62F,B00Z9K379C,759387370,ANWING Waterproof Wireless Bluetooth Speaker O...,Mobile_Electronics,4,0.0,2.0,N,Y,Four Stars,"this is a very good item, it provide me the so...",2015-08-31,199.113.191.75
9,US,48701722,RBEMQ29WJBHYG,B00YO3UYXW,913911909,Jensen MCR-100 Cassette Player/Recorder 1 Touc...,Mobile_Electronics,5,164.0,168.0,N,Y,I LOVE my recorder,I LOVE my recorder. Bought it obviously becaus...,2015-08-31,194.248.209.55


In [103]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import nltk
import re
from nltk.corpus import stopwords 
import collections
from nltk.corpus import wordnet

In [106]:
dataset.set_index("review_id",inplace=True)

In [112]:
# Latent symantic analysis
# it will analyse all reviews and determine all reviews belong to the same concept
def LSA(text):
    #text is list of reviews of same product
    
    
    # Created TF-IDF Model
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(text)
    
    # Created SVD(Singular Value Decomposition)
    lsa = TruncatedSVD(n_components = 1,n_iter = 100)
    lsa.fit(X)
    
    terms = vectorizer.get_feature_names()
    concept_words={}

    for j,comp in enumerate(lsa.components_):
        componentTerms = zip(terms,comp)
        sortedTerms = sorted(componentTerms,key=lambda x:x[1],reverse=True)
        sortedTerms = sortedTerms[:10]
        concept_words[str(j)] = sortedTerms
     
    sentence_scores = []
    for key in concept_words.keys():
        for sentence in text:
            words = nltk.word_tokenize(sentence)
            scores = 0
            for word in words:
                for word_with_scores in concept_words[key]:
                    if word == word_with_scores[0]:
                        scores += word_with_scores[1]
            sentence_scores.append(scores)
    return sentence_scores

In [ ]:
product_df = dataset.groupby("product_id")
#grouping dataset by product_id

unique_products = dataset["product_id"].unique()
#stores list of all product id

no_products = len(unique_products.tolist())
#no. of products

remove_reviews = []
#store review_id that are fake 
    
for i in range(no_products):
    #iterate through all product reviews 
    
    df = product_df.get_group(unique_products[i])
    #dataframe of a single product
    
    unique_reviews = df.index.tolist()
    #list of review_id of reviews of same product
    
    no_reviews = len(unique_reviews)   
    #no. of reviews of same product
    
    count = no_reviews 
    #count is no. of reviews that can be analysed
    
    reviews = []
    #list of review texts
    
    review_id = []
    #list of review texts
    
    for j in range(no_reviews):
        #iterate through all reviews
        
        text = str(df.loc[unique_reviews[j]]["review_body"])
        # text of a review 
        
        #cleaning the text
        text = re.sub(r"\W"," ",text)             
        text = re.sub(r"\d"," ",text)             
        text = re.sub(r"\s+[a-z]\s+"," ",text)    
        text = re.sub(r"^[a-z]\s+"," ",text)    
        text = re.sub(r"\s+[a-z]$"," ",text)    
        text = re.sub(r"\s+"," ",text)
        
        words = nltk.word_tokenize(text)
        #text into word list
        
        if(len(words)==1):
        #if only one word in text review
            
            if(len(text) <=1 or not wordnet.synsets(text) ):
            #if word is having only one character or invalid english word
                
                remove_reviews.append(unique_reviews[j])
                #append this review as fake
                
                count-=1
                #review left to be analysed will be decrease by 1
                
                continue
                #check for next review
                '''
            elif(len(words[0])<=1):
                remove_reviews.append(unique_reviews[j])
                count-=1
                continue
                '''
        elif(len(words)==0):
        #if no words
            
            remove_reviews.append(unique_reviews[j])
            #append this review as fake
            
            count-=1
            #review left to be analysed will be decrease by 1
            
            continue
            #check for next review
        
        review_id.append(unique_reviews[j])
        #valid: append review_id to review_id list for analysis
        
        reviews.append(text)
        #valid: append review_body to reviews list for analysis
        
    ###########################################################################################
    if(count<=0):
        #if no reviews left to analyse 

        continue
        #check for next
        
    if(count==1): 
        #if only one review is left to analyse
        
        #check concept between product title and the review
        
        text = [text,str(df.loc[review_id[0]]["product_title"])] 
        #add product_title and review to the list 
        
        sentence_scores = LSA(text) 
        #call LSA method to get the score
        
        if(sentence_scores[0]==0): 
        #if review score is 0, then it's fake
            remove_reviews.append(review_id[0])
        continue
    
    #list of scores of reviews of same product
    sentence_scores = LSA(reviews)
            
    for j in range(len(sentence_scores)):
        #iterating through all the scores
        
        if(sentence_scores[j]==0.00):
            # if score is 0, it's fake
            remove_reviews.append(review_id[j])